In [2]:
from bs4 import BeautifulSoup
import json
import requests
from pathlib import Path
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
api_key  = os.getenv('OPENAI_APIKEY')
subscription_key = os.getenv('BING_APIKEY')
search_engine=os.getenv('SEARCH_ENGINE')
assert subscription_key

def get_web_content(sentence, num_results=1):

    headers = {
    }
    params = {
        "key": subscription_key,
        "cx": search_engine,
        "q": sentence,
    }
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results =response.json()

    if 'items' in search_results:
        web_pages = search_results['items']
        urls = [page['formattedUrl'] for page in web_pages]

    all_text = ""
    print(urls)
    for url in urls:
        response = requests.get(url)

        html_content = response.content.decode('utf-8')

        soup = BeautifulSoup(html_content, 'html.parser')
        body = soup.find('body')

        if body:
            body_text = body.get_text(separator=' ', strip=True)
            all_text += body_text + "\n\n"
        else:
            print(f"No <body> tag found for URL: {url}")

    if all_text:
        message = f"以下は「{sentence}」に関する情報です。正確な情報を取得してください。\n{all_text}"
        completion = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "user", "content": message}
            ]
        )

        return completion.choices[0].message.content
    else:
        return "No text was extracted from the provided URLs."

client = OpenAI(
    api_key=api_key,
)
search_url = "https://www.googleapis.com/customsearch/v1"

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_web_content",
            "description": "引数の文章でweb上を検索して正確な情報を返す関数。",
            "parameters": {
                "type": "object",
                "properties": {
                    "sentence": {
                        "type": "string",
                        "description": "検索する文章。",
                    },
                },
                "required": ["sentence"],
            },
        },
    }
]

messages = [
    {"role": "system", "content": "あなたは優秀なアシスタントです。聞かれた情報を学習していない場合はweb上を検索して正確な情報を返します。"}
]

while True:
    user_input = input("\nあなたの質問を入力してください(終了するには'quit'と入力):")
    if user_input.lower() == "quit":
        break

    messages.append({"role":"user", "content": user_input})

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )

    assistant_response = completion.choices[0].message
    tool_calls = assistant_response.tool_calls
    print(tool_calls)

    if tool_calls:
        tool_calls_list = []
        for tool_call in tool_calls:
            tool_call_id = tool_call.id
            tool_call_type = tool_call.type
            tool_call_function = tool_call.function

            tool_calls_list.append({
                "id": tool_call_id,
                "type": tool_call_type,
                "function": {
                    "name": tool_call_function.name,
                    "arguments": tool_call_function.arguments
                }
            })

        messages.append({
            "role": "assistant",
            "tool_calls": tool_calls_list
        })

        for tool_call in tool_calls_list:
            print(tool_call['function']['name'])
            if tool_call['function']['name'] == "get_web_content":
                arguments = json.loads(tool_call['function']["arguments"])
                sentence = arguments['sentence']
                web_content = get_web_content(sentence, num_results=1)
                messages.append({"tool_call_id": tool_call['id'], "role": "tool", "name": tool_call['function']['name'], "content": web_content})

        # すべての変更をAPIに送信して結果を取得
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        assistant_response = completion.choices[0].message.content
        print("Assistant:", assistant_response)
        messages.append({"role": "assistant", "content": assistant_response})
    else:
        assistant_response = assistant_response.content
        print("Assistant:", assistant_response)
        messages.append({"role": "assistant", "content": assistant_response})

None
Assistant: Hello! How can I assist you today?
